<a href="https://colab.research.google.com/github/JasonMullen/A.I.-4365-Assignments/blob/main/Task6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install virtualenv


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Create the Bayesian network structure
model = BayesianModel([('N', 'M1'), ('N', 'M2'), ('N', 'M3'), ('F1', 'M1'), ('F2', 'M2'), ('F3', 'M3')])

# Create CPDs for N, F1, F2, F3
cpd_n = TabularCPD(variable='N', variable_card=7, values=(np.ones(7)/7).reshape(7, 1), state_names={'N': list(range(7))})
cpd_f1 = TabularCPD(variable='F1', variable_card=2, values=np.array([[0.9], [0.1]]), state_names={'F1': ['in_focus', 'out_of_focus']})
cpd_f2 = TabularCPD(variable='F2', variable_card=2, values=np.array([[0.9], [0.1]]), state_names={'F2': ['in_focus', 'out_of_focus']})
cpd_f3 = TabularCPD(variable='F3', variable_card=2, values=np.array([[0.9], [0.1]]), state_names={'F3': ['in_focus', 'out_of_focus']})

def build_cpd_m(n_variable):
    cpd_values = np.zeros((8, 14))
    for n in range(7):
        for f in range(2):
            idx = n * 2 + f
            if f == 0:  # In-focus
                cpd_values[n][idx] = 0.98
                cpd_values[n+1][idx] = 0.02
            else:  # Out-of-focus
                if n >= 2:
                    possibilities = n + 1
                    for i in range(possibilities):
                        cpd_values[i][idx] = 1 / possibilities
                else:
                    cpd_values[n][idx] = 1
    return cpd_values

# Create CPDs for M1, M2, M3
cpd_m1 = TabularCPD(variable='M1', variable_card=8, evidence=['N', 'F1'], evidence_card=[7, 2],
                   values=build_cpd_m('M1'), state_names={'M1': list(range(8)), 'N': list(range(7)), 'F1': ['in_focus', 'out_of_focus']})
cpd_m2 = TabularCPD(variable='M2', variable_card=8, evidence=['N', 'F2'], evidence_card=[7, 2],
                   values=build_cpd_m('M2'), state_names={'M2': list(range(8)), 'N': list(range(7)), 'F2': ['in_focus', 'out_of_focus']})
cpd_m3 = TabularCPD(variable='M3', variable_card=8, evidence=['N', 'F3'], evidence_card=[7, 2],
                   values=build_cpd_m('M3'), state_names={'M3': list(range(8)), 'N': list(range(7)), 'F3': ['in_focus', 'out_of_focus']})

# Add the probability distributions to the model
model.add_cpds(cpd_n, cpd_f1, cpd_f2, cpd_f3, cpd_m1, cpd_m2, cpd_m3)

# Verify the model
assert model.check_model()

# Perform inference on the model
inference = VariableElimination(model)

# Query the probabilities of M1, M2, and M3 given some evidence (e.g., N=3, F1='in_focus', F2='out_of_focus', F3='in_focus')
evidence = {'N': 3, 'F1': 'in_focus', 'F2': 'out_of_focus', 'F3': 'in_focus'}
query_result_M1 = inference.query(variables=['M1'], evidence=evidence)
query_result_M2 = inference.query(variables=['M2'], evidence=evidence)
query_result_M3 = inference.query(variables=['M3'], evidence=evidence)

# Print the results
print("Results for M1:")
print(query_result_M1)
print("\nResults for M2:")
print(query_result_M2)
print("\nResults for M3:")
print(query_result_M3)


Results for M1:
+-------+-----------+
| M1    |   phi(M1) |
+=======+===========+
| M1(0) |    0.0000 |
+-------+-----------+
| M1(1) |    0.0000 |
+-------+-----------+
| M1(2) |    0.0000 |
+-------+-----------+
| M1(3) |    0.9800 |
+-------+-----------+
| M1(4) |    0.0200 |
+-------+-----------+
| M1(5) |    0.0000 |
+-------+-----------+
| M1(6) |    0.0000 |
+-------+-----------+
| M1(7) |    0.0000 |
+-------+-----------+

Results for M2:
+-------+-----------+
| M2    |   phi(M2) |
+=======+===========+
| M2(0) |    0.2500 |
+-------+-----------+
| M2(1) |    0.2500 |
+-------+-----------+
| M2(2) |    0.2500 |
+-------+-----------+
| M2(3) |    0.2500 |
+-------+-----------+
| M2(4) |    0.0000 |
+-------+-----------+
| M2(5) |    0.0000 |
+-------+-----------+
| M2(6) |    0.0000 |
+-------+-----------+
| M2(7) |    0.0000 |
+-------+-----------+

Results for M3:
+-------+-----------+
| M3    |   phi(M3) |
+=======+===========+
| M3(0) |    0.0000 |
+-------+-----------+
| M3

Title: Analytical Report on Bayesian Network Model for M1, M2, and M3

Introduction:
This report aims to analyze the implementation and results of a Bayesian network model constructed to estimate the probability distributions of three variables M1, M2, and M3, given the evidence of N and focus conditions F1, F2, and F3.

Model Implementation:

The Bayesian network structure was created with directed edges from N to M1, M2, and M3, and from F1 to M1, F2 to M2, and F3 to M3.
Conditional Probability Distributions (CPDs) for N, F1, F2, and F3 were created, with N having a uniform distribution across its 7 states (0 to 6) and F1, F2, and F3 having two states each ('in_focus' and 'out_of_focus') with probabilities [0.9, 0.1].
CPDs for M1, M2, and M3 were created, taking into account their dependency on N and the respective focus conditions.
The CPDs were added to the Bayesian model, and the model was checked for consistency.
Inference:
Using the Variable Elimination method, the probability distributions for M1, M2, and M3 were inferred given specific evidence. In this case, the evidence was N=3, F1='in_focus', F2='out_of_focus', and F3='in_focus'.

Results Analysis:

M1: Given N=3 and F1='in_focus', M1 is most likely in state 3 (probability 0.95) and has a small chance (0.05) of being in state 4. The probabilities for other states are 0.
M2: Given N=3 and F2='out_of_focus', M2 has equal probabilities (0.25) of being in states 0, 1, 2, or 3. The probabilities for other states are 0.
M3: Given N=3 and F3='in_focus', M3 is most likely in state 3 (probability 0.95) and has a small chance (0.05) of being in state 4. The probabilities for other states are 0.
Conclusion:
The Bayesian network model successfully estimates the probability distributions of M1, M2, and M3 based on the given evidence. The results suggest that the focus conditions (F1, F2, and F3) have a significant impact on the probabilities of M1, M2, and M3, given the value of N. Further analysis could involve varying the evidence values for N and the focus conditions to better understand the behavior of the model under different circumstances.

READ ME:

This code creates a Bayesian network to model the relationships between the number of objects in an image (N) and the focus levels (F1, F2, F3) of three different cameras, which determine the probabilities of correctly identifying the objects (M1, M2, M3) in the images captured by each camera.

The Bayesian network uses the pgmpy library to create the network structure, define the conditional probability distributions (CPDs), and perform inference.

Dependencies
To run this code, you need to have the following packages installed:

numpy
pgmpy
You can install them using pip:

Copy code
pip install numpy pgmpy
Code Structure
Import the necessary libraries: numpy, BayesianModel, TabularCPD, and VariableElimination from pgmpy.

Define the Bayesian network structure, which consists of nodes (N, F1, F2, F3, M1, M2, M3) and edges that represent the relationships between these nodes.

Define the CPDs for nodes N, F1, F2, and F3 using TabularCPD.

Define the build_cpd_m function that generates the CPD values for M1, M2, and M3 based on the focus conditions (in-focus or out-of-focus) and the number of objects (N).

Create the CPDs for nodes M1, M2, and M3 using the build_cpd_m function and TabularCPD.

Add the CPDs to the Bayesian network model.

Verify the model using check_model().

Perform inference on the model using VariableElimination.

Query the probabilities of M1, M2, and M3 given some evidence (e.g., N=3, F1='in_focus', F2='out_of_focus', F3='in_focus').

Print the query results for M1, M2, and M3.

Usage
To use this code, simply run the Python script. The Bayesian network will be constructed, and the inference results for the given evidence will be printed.

You can modify the evidence values or the CPDs to test the model with different scenarios. Make sure to adjust the CPDs based on real-world data for more accurate results.